In [32]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Reshape, GlobalAveragePooling2D, Activation,UpSampling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

from mediapipe.python.solutions import pose as mp_pose
import cv2
pose_tracker = mp_pose.Pose()

In [77]:
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN_landmark/'
valid_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID_landmark/'
test_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST_landmark/'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32,shuffle = False)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32,shuffle = False)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 7135 images belonging to 15 classes.
Found 2518 images belonging to 15 classes.
Found 1223 images belonging to 15 classes.


**Data Preprocessing**

In [102]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.mobilenet import preprocess_input
from mediapipe.python.solutions import pose as mp_pose
import numpy as np
from tensorflow import keras

class LandmarkDataGenerator(keras.utils.Sequence):
    def __init__(self, image_paths, class_labels, batch_size=32, target_size=(224, 224)):
        self.image_paths = image_paths
        self.class_labels = class_labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.pose_tracker = mp_pose.Pose()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_class_labels = self.class_labels[index * self.batch_size:(index + 1) * self.batch_size]

        images = []
        landmarks = []

        for image_path in batch_image_paths:
            image = cv2.imread(image_path)
            images.append(image)

            # Process the image to extract landmarks
            results = self.pose_tracker.process(image)
            pose_landmarks = results.pose_landmarks

            if pose_landmarks is not None:
                landmark_coordinates = []
                for landmark in pose_landmarks.landmark:
                    landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
                landmarks.append(landmark_coordinates)
            else:
                landmarks.append([])

        images = np.array(images)
        landmarks = np.array(landmarks, dtype=object)
        class_labels = np.array(batch_class_labels)

        return [images, landmarks], class_labels





In [103]:

train_data_generator = LandmarkDataGenerator(train_batches.filepaths, train_batches.classes)

# # Create the custom data generator for validation
# valid_data_generator = LandmarkDataGenerator(valid_image_paths, valid_class_labels)

# # Create the custom data generator for testing
# test_data_generator = LandmarkDataGenerator(test_image_paths, test_class_labels)

# # Example of iterating over the data generator
# for [images, landmarks], class_labels in train_data_generator:
#     # Perform model training or evaluation using the images, landmarks, and class labels
#     ...
train_batches.classes

array([ 0,  0,  0, ..., 12, 12, 12])

In [91]:
for i in range(5):
    image_path = train_batches.filepaths[i]
    print(f"Processing image: {image_path}")

    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        continue

    ([images, landmarks], class_labels) = train_data_generator[i]

    image_filename = os.path.basename(image_path)
    landmark = landmarks[i]
    clas = train_data_generator.class_labels[i]

    print(f"Image Filename: {image_filename}")
    print(f"Landmark: {landmark}")
    print(f"Class: {clas}")
    print()


Processing image: C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN_landmark/calling\Image_10023.jpg


C:\Users\PRASAN~1\AppData\Local\Temp/ipykernel_16844/3177914268.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  landmarks = np.array(landmarks)


Image Filename: Image_10023.jpg
Landmark: [[0.17204752564430237, 0.3809846341609955, -0.5439518094062805], [0.17611679434776306, 0.3525908887386322, -0.5067144632339478], [0.18170447647571564, 0.35061001777648926, -0.5069411993026733], [0.1867808848619461, 0.34895089268684387, -0.507157564163208], [0.1561826765537262, 0.358000248670578, -0.5149974226951599], [0.14691226184368134, 0.36029234528541565, -0.515192985534668], [0.13801103830337524, 0.36267486214637756, -0.5153151154518127], [0.18534205853939056, 0.354740172624588, -0.28072836995124817], [0.1190120205283165, 0.3722021281719208, -0.3175814151763916], [0.18184855580329895, 0.4012134373188019, -0.4584673047065735], [0.157296821475029, 0.40613752603530884, -0.4695717692375183], [0.24361467361450195, 0.4950238764286041, -0.16085274517536163], [0.07621230185031891, 0.5090106129646301, -0.22806771099567413], [0.2784374952316284, 0.6441208720207214, -0.08707142621278763], [0.10636115074157715, 0.6696822047233582, -0.48280081152915955

In [92]:
import numpy as np

# Initialize an empty list to store the class labels
all_class_labels = []

# Iterate over the generator and collect the class labels
for _, labels in train_data_generator:
    all_class_labels.extend(labels)

# Convert the list to a NumPy array
all_class_labels = np.array(all_class_labels)

# Count the unique classes
num_classes = len(np.unique(all_class_labels))

# Print the total number of classes
print(f"Total number of classes: {num_classes}")


C:\Users\PRASAN~1\AppData\Local\Temp/ipykernel_16844/3177914268.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  landmarks = np.array(landmarks)


Total number of classes: 13


**Data Prcoessing 2**

In [36]:
# import numpy as np
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from mediapipe.python.solutions import pose as mp_pose

# import os
# import cv2
# import numpy as np

# main_dir = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN_landmark/'

# landmarks_list = []  # List to store landmarks for all images
# image_paths = []  # List to store image paths

# # Create the pose tracker
# pose_tracker = mp_pose.Pose()

# # Iterate through the subdirectories in the main directory
# for subdir in os.listdir(main_dir):
#     subdir_path = os.path.join(main_dir, subdir)
#     if os.path.isdir(subdir_path):
#         # Iterate through the images in the subdirectory
#         for filename in os.listdir(subdir_path):
#             if filename.endswith(".jpg") or filename.endswith(".png"):  # Add any other valid image extensions if needed
#                 image_path = os.path.join(subdir_path, filename)
#                 image = cv2.imread(image_path)
#                 results = pose_tracker.process(image)
#                 landmarks = results.pose_landmarks

#                 if landmarks is not None:
#                     landmark_coordinates = []
#                     for landmark in landmarks.landmark:
#                         landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
#                     landmarks_list.append(landmark_coordinates)
#                     image_paths.append(image_path)
#                 else:
#                     # Remove the image if landmarks are not found
#                     os.remove(image_path)

# # Convert the landmarks list to a numpy array
# landmarks_array = np.array(landmarks_list)

# # Save the landmarks array as a numpy file
# np.save('train_landmarks.npy', landmarks_array)



In [37]:
# landmarks_array.shape

In [38]:
# import numpy as np
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from mediapipe.python.solutions import pose as mp_pose

# import os
# import cv2
# import numpy as np

# main_dir = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID_landmark/'

# landmarks_list = []  # List to store landmarks for all images
# image_paths = []  # List to store image paths

# # Create the pose tracker
# pose_tracker = mp_pose.Pose()

# # Iterate through the subdirectories in the main directory
# for subdir in os.listdir(main_dir):
#     subdir_path = os.path.join(main_dir, subdir)
#     if os.path.isdir(subdir_path):
#         # Iterate through the images in the subdirectory
#         for filename in os.listdir(subdir_path):
#             if filename.endswith(".jpg") or filename.endswith(".png"):  # Add any other valid image extensions if needed
#                 image_path = os.path.join(subdir_path, filename)
#                 image = cv2.imread(image_path)
#                 results = pose_tracker.process(image)
#                 landmarks = results.pose_landmarks

#                 if landmarks is not None:
#                     landmark_coordinates = []
#                     for landmark in landmarks.landmark:
#                         landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
#                     landmarks_list.append(landmark_coordinates)
#                     image_paths.append(image_path)
#                 else:
#                     # Remove the image if landmarks are not found
#                     os.remove(image_path)

# # Convert the landmarks list to a numpy array
# landmarks_array_valid = np.array(landmarks_list)

# # Save the landmarks array as a numpy file
# np.save('valid_landmarks.npy', landmarks_array_valid)

In [39]:
# landmarks_array_valid.shape

In [40]:
# import numpy as np
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from mediapipe.python.solutions import pose as mp_pose

# import os
# import cv2
# import numpy as np

# main_dir = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST_landmark/'

# landmarks_list = []  # List to store landmarks for all images
# image_paths = []  # List to store image paths

# # Create the pose tracker
# pose_tracker = mp_pose.Pose()

# # Iterate through the subdirectories in the main directory
# for subdir in os.listdir(main_dir):
#     subdir_path = os.path.join(main_dir, subdir)
#     if os.path.isdir(subdir_path):
#         # Iterate through the images in the subdirectory
#         for filename in os.listdir(subdir_path):
#             if filename.endswith(".jpg") or filename.endswith(".png"):  # Add any other valid image extensions if needed
#                 image_path = os.path.join(subdir_path, filename)
#                 image = cv2.imread(image_path)
#                 results = pose_tracker.process(image)
#                 landmarks = results.pose_landmarks

#                 if landmarks is not None:
#                     landmark_coordinates = []
#                     for landmark in landmarks.landmark:
#                         landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
#                     landmarks_list.append(landmark_coordinates)
#                     image_paths.append(image_path)
#                 else:
#                     # Remove the image if landmarks are not found
#                     os.remove(image_path)

# # Convert the landmarks list to a numpy array
# landmarks_array_test = np.array(landmarks_list)

# # Save the landmarks array as a numpy file
# np.save('test_landmarks.npy', landmarks_array_test)

In [41]:
# landmarks_array_test.shape

In [21]:
import numpy as np

# Get the landmarks for each image in train_batches
landmarks_list = []
for i in range(len(train_batches.filenames)):
    image_path = train_batches.filepaths[i]
    image = cv2.imread(image_path)
    results = pose_tracker.process(image)
    landmarks = results.pose_landmarks

    if landmarks is not None:
        landmark_coordinates = []
        for landmark in landmarks.landmark:
            landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
        landmarks_list.append(landmark_coordinates)

# Convert the landmarks list to a numpy array
landmarks_array = np.array(landmarks_list)

In [42]:
print(landmarks_array.shape)


(7135, 33, 3)


In [ ]:
np.save('train_landmarks.npy', landmarks_array)

In [ ]:

import os

# Function to check if an image has landmarks
def has_landmarks(image_path):
    image = cv2.imread(image_path)
    results = pose_tracker.process(image)
    landmarks = results.pose_landmarks
    return landmarks is not None

# Iterate over the main directory and its subfolders
for root, dirs, files in os.walk(train_path):
    # Iterate over the subfolders
    for subdir in dirs:
        subfolder_path = os.path.join(root, subdir)
        
        # Iterate over the image files in the subfolder
        for file in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, file)
            
            # Check if the image has landmarks
            if not has_landmarks(image_path):
                # Delete the image
                os.remove(image_path)
                print("Deleted:", image_path)



In [54]:
import numpy as np

# # Get the landmarks for each image in train_batches
# landmarks_list = []
# for i in range(len(valid_batches.filenames)):
#     image_path = valid_batches.filepaths[i]
#     image = cv2.imread(image_path)
#     results = pose_tracker.process(image)
#     landmarks = results.pose_landmarks

#     if landmarks is not None:
#         landmark_coordinates = []
#         for landmark in landmarks.landmark:
#             landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
#         landmarks_list.append(landmark_coordinates)

# # Convert the landmarks list to a numpy array
# landmarks_array = np.array(landmarks_list)

In [55]:
# np.save('valid_landmarks.npy', landmarks_array)

In [56]:
# import numpy as np

# # Get the landmarks for each image in train_batches
# landmarks_list = []
# for i in range(len(test_batches.filenames)):
#     image_path = test_batches.filepaths[i]
#     image = cv2.imread(image_path)
#     results = pose_tracker.process(image)
#     landmarks = results.pose_landmarks

#     if landmarks is not None:
#         landmark_coordinates = []
#         for landmark in landmarks.landmark:
#             landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
#         landmarks_list.append(landmark_coordinates)

# # Convert the landmarks list to a numpy array
# landmarks_array = np.array(landmarks_list)

In [57]:
# np.save('test_landmarks.npy', landmarks_array)

**Model building**

In [11]:
import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
])

# Build the model
model.build((None,224, 224,3))
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 1280)              4226432   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               163968    
Total params: 4,390,400
Trainable params: 4,366,000
Non-trainable params: 24,400
_________________________________________________________________


In [12]:
from tensorflow.keras.layers import Input
landmarks_input = Input(shape=(None,33, 3))

mobile1 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape = (None,33,3),weights = None,include_top = False)
mobile1.summary()

Model: "mobilenetv2_1.00_None"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 33, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, 17, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, 17, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, 17, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________

In [13]:
from tensorflow.keras.layers import GlobalAveragePooling2D

x = mobile1.layers[-1].output

x = GlobalAveragePooling2D()(x)  # Global Average Pooling to reduce dimensions to (None, 1)
x = Dropout(0.1)(x)
output = Dense(128, activation='relu')(x)


model1 = Model(inputs=mobile1.input, outputs=output)
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 33, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, 17, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, 17, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, 17, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate, Dense

combined = concatenate([model.output,model1.output,])
fin = Dense(15, activation='softmax')(combined)

fin_model = Model(inputs=[model.input,model1.input], outputs=fin)
fin_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 33, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, 17, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, 17, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, 17, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [46]:
# Assuming the model expects two inputs: image input and landmark input
image_input = train_batches.next()[0]  
print(image_input.shape)

(32, 224, 224, 3)


In [104]:

# Assuming the model expects two inputs: image input and landmark input
# image_input = train_batches.next()[0]  # Get the next batch of images from train_batches
landmark_input = landmarks_array

# Compile the model
fin_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
fin_model.fit(x=train_data_generator, epochs=15)





ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).